In [21]:
gold_folder_path = 'abfss://nyc-taxi-data@nycanalyticsmarlabsdl.dfs.core.windows.net/gold'
silver_folder_path = 'abfss://nyc-taxi-data@nycanalyticsmarlabsdl.dfs.core.windows.net/silver'
gold_folder_path = 'abfss://nyc-taxi-data@nycanalyticsmarlabsdl.dfs.core.windows.net/gold'

StatementMeta(nycsparkpool, 0, 22, Finished, Available, Finished)

In [22]:
spark.conf.set("spark.sql.sources.partitionColumnTypeInfrence.enabled", "false")

StatementMeta(nycsparkpool, 0, 23, Finished, Available, Finished)

In [23]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc_taxi_ldw_spark
LOCATION 'abfss://nyc-taxi-data@nycanalyticsmarlabsdl.dfs.core.windows.net/gold'

StatementMeta(nycsparkpool, 0, 24, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [24]:
trip_data_green_df = spark.read.parquet(f"{silver_folder_path}/trip_data_green")
trip_data_green_df

StatementMeta(nycsparkpool, 0, 25, Finished, Available, Finished)

DataFrame[VendorID: int, lpep_pickup_datetime: timestamp, lpep_dropoff_datetime: timestamp, store_and_fwd_flag: string, RatecodeID: int, PULocationID: int, DOLocationID: int, passenger_count: int, trip_distance: double, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, ehail_fee: int, improvement_surcharge: double, total_amount: double, payment_type: int, trip_type: int, congestion_surcharge: double, year: int, month: int]

In [25]:
# total trip count
# total fare amount

from pyspark.sql.functions import *



trip_data_green_agg_df = trip_data_green_df \
                         .groupBy("year", "month", "PULocationID", "DOLocationID") \
                         .agg(count(lit(1)).alias("total_trip_count"),
                         round(sum("fare_amount"), 2).alias("total_fare_amount"))

StatementMeta(nycsparkpool, 0, 26, Finished, Available, Finished)

In [26]:
trip_data_green_agg_df.write.mode("overwrite").partitionBy("year", "month") \
.format("parquet").saveAsTable("nyc_taxi_ldw_spark.trip_data_green_agg")

StatementMeta(nycsparkpool, 0, 27, Finished, Available, Finished)